<a href="https://colab.research.google.com/github/JupaaF/Proyecto_Final_PINNs/blob/main/Arquitectura_y_entrenamiento.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import torch
import torch.nn as nn
import numpy as np

In [2]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"{device}")

cpu


Lo que esta abajo es el codigo de Lucas, lo copie para probarlo y ver si anda etc etc. No hace metricas relacionando con los valores reales, ni hace test. Hay varias cosas por mejorar desde el lado de Ingeniero de IA digamos. Otra cosa que tenemos uqe verificar es si tomamos el modelo este, y le cambiamos las condiciones de borde, cuanto error genera en las nuevas predicciones si no lo volves a entrenar.

In [3]:
######################################################################################
# PINNs para problemas de fluidos. Canal Tesis doctoral Scacchi
# El codigo aplica PINNs al ensayo de laboratorio de la tesis de Scacchi implementado hidrodinamicay morfología
# Utiliza 3 capas de 50 neuronas, 7000 epocas, y optimizador ADAM
# Dominguez Ruben Lucas. FICH-UNL
######################################################################################

# Importamos librerías
import torch
import torch.nn as nn
import numpy as np
import matplotlib.pyplot as plt
#######################################################################################
# START del código
# Sirve para quienes tengan CUDA (usarlo en Google Colab)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using:", device)
######################################################################################
# Parametros geometricos del canal
######################################################################################
L, B = 12.0, 1.48
B_half = B / 2
Q = 0.0837  # Discharge (m^2/s)
h_left = 0.193  # Inlet depth (m, from uniform flow)
h_right = 0.184  # Outlet depth (m)
g = 9.81  # Gravity (m/s^2)
n = 0.019  # Manning's n
S_0 = 0.00072  # Bed slope (no utilizable porque el fondo es plano)
eps = 1e-6  # Avoid division by zero


##################################################################################

# Meyer-Peter Müller bedload transport. Formulación de transporte de fondo (Puede utilizarse otras VanRijn, Engelund Hansen, etc)
def q_b(h, u):
    tau = h * u**2 + eps
    tau_c = 0.047
    return torch.where(tau > tau_c, 8 * (tau - tau_c).clamp(min=0)**1.5, torch.zeros_like(tau))
#################################
# Red
#################################
class PINN2D(nn.Module):
    def __init__(self):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(2, 50),
            nn.Tanh(),
            nn.Linear(50, 50),
            nn.Tanh(),
            nn.Linear(50, 50),
            nn.Tanh(),
            nn.Linear(50, 3)  # h, u, zb
        )

    def forward(self, x):
        out = self.net(x)
        h = torch.nn.functional.softplus(out[:, 0:1]) + 0.1  # Ensure h > 0.1
        u = out[:, 1:2]  # Allow negative velocities
        zb = out[:, 2:3]  # Flat bed: zb = 0 initially, learned by the network
        return torch.cat([h, u, zb], dim=1)

# Normalization
def normalize(x, xmin, xmax):
    return 2 * (x - xmin) / (xmax - xmin) - 1

# Funcion de peridoda de la continuidad (constraint)
def loss_function(model, xy_colloc, xy_bc_left, xy_bc_right, xy_bc_wall, xy_bc_con):
    xy_colloc.requires_grad = True
    huzb = model(xy_colloc)
    h, u, zb = huzb[:, 0:1], huzb[:, 1:2], huzb[:, 2:3]

    # Gradients
    h_grad = torch.autograd.grad(h, xy_colloc, grad_outputs=torch.ones_like(h), create_graph=True)[0]
    u_grad = torch.autograd.grad(u, xy_colloc, grad_outputs=torch.ones_like(u), create_graph=True)[0]
    zb_grad = torch.autograd.grad(zb, xy_colloc, grad_outputs=torch.ones_like(zb), create_graph=True)[0]

    h_x, h_y = h_grad[:, 0:1], h_grad[:, 1:2]
    u_x, u_y = u_grad[:, 0:1], u_grad[:, 1:2]
    zb_x = zb_grad[:, 0:1]

    # Friction slope
    Sf = (n**2 * u**2) / (h.clamp(min=0.05)**(4/3))

    # PDE residuals
    mass_res = h * u - Q / B  # Continuity (v = 0)
    mom_x = u * u_x + g * h_x + g * Sf + g * zb_x  # x-momentum, no S_0 (flat bed)

    # Exner equation (cambio del fondo)
    qb = q_b(h, u)
    qb_grad = torch.autograd.grad(qb, xy_colloc, grad_outputs=torch.ones_like(qb), create_graph=True)[0]
    dqb_dx = qb_grad[:, 0:1]
    zb_t = -dqb_dx / (1 - 0.4)

    # Constriction: zero bed evolution (esto es para la zona del estribo)
    xy_bc_con.requires_grad = True
    huzb_con = model(xy_bc_con)
    h_con, u_con, zb_con = huzb_con[:, 0:1], huzb_con[:, 1:2], huzb_con[:, 2:3]
    qb_con = q_b(h_con, u_con)
    qb_con_grad = torch.autograd.grad(qb_con, xy_bc_con, grad_outputs=torch.ones_like(qb_con), create_graph=True)[0]
    dqb_dx_con = qb_con_grad[:, 0:1]
    zb_t_con = -dqb_dx_con / (1 - 0.4)

    # Boundary conditions
    huzb_left = model(xy_bc_left)
    huzb_right = model(xy_bc_right)
    huzb_wall = model(xy_bc_wall)

    loss_bc = torch.mean((huzb_left[:, 0] - h_left)**2)  # Inlet: h = h_left
    loss_bc += torch.mean((huzb_left[:, 0] * huzb_left[:, 1] - Q / B)**2)  # Inlet: h*u = Q/B
    loss_bc += torch.mean((huzb_right[:, 0] - h_right)**2)  # Outlet: h = h_right
    loss_bc += torch.mean(huzb_wall[:, 1]**2)  # Walls: u = 0
    loss_bc += torch.mean(huzb_con[:, 1]**2)  # Constriction: u = 0
    loss_bc += torch.mean(zb_t_con**2)  # Constriction: zb_t = 0

    # Flow continuity constraint: ensure Q = \int h*u dy across sections
    N_y = 50
    x_sections = torch.tensor([0.0, 3.0, 6.0, 9.0, 12.0], device=device)
    loss_Q = 0.0
    y_cross = torch.linspace(-B_half, B_half, N_y, device=device)
    dy = B / (N_y - 1)

    for x_sec in x_sections:
        x_cross = torch.full((N_y,), x_sec, device=device)
        xy_cross = torch.stack([x_cross, y_cross], dim=1)
        xy_cross_norm = normalize(xy_cross, torch.tensor([0, -B_half], device=device), torch.tensor([L, B_half], device=device))
        pred_cross = model(xy_cross_norm)
        h_cross, u_cross = pred_cross[:, 0], pred_cross[:, 1]
        hu = h_cross * u_cross
        Q_calc = torch.trapz(hu, dx=dy)
        loss_Q += (Q_calc - Q)**2

    loss_Q = loss_Q / len(x_sections)

    return (
        torch.mean(mass_res**2) +
        10 * torch.mean(mom_x**2) +
        torch.mean(zb_t**2) +
        100 * loss_bc +
        100 * loss_Q
    )

# Malla
N = 50 #(el numero de colocaciones puede reducirse)
x = torch.linspace(0, L, N)
y = torch.linspace(-B_half, B_half, N)
X, Y = torch.meshgrid(x, y, indexing='ij')
xy_colloc = torch.stack([X.flatten(), Y.flatten()], dim=1).to(device)

# Boundary points
xy_bc_left = torch.stack([torch.zeros(N), y], dim=1).to(device)
xy_bc_right = torch.stack([torch.full((N,), L), y], dim=1).to(device)
xy_bc_wall = torch.cat([
    torch.stack([x, torch.full_like(x, -B_half)], dim=1),
    torch.stack([x, torch.full_like(x, B_half)], dim=1)
], dim=0).to(device)

# Posicionamiento del estribo (x = 2.94 to 3.06, y = -0.44 to -0.74)
N_con = 50
x_con = torch.linspace(2.94, 3.06, N_con)
y_con = torch.linspace(-0.44, -0.74, N_con)
X_con, Y_con = torch.meshgrid(x_con, y_con, indexing='ij')
xy_bc_con = torch.stack([X_con.flatten(), Y_con.flatten()], dim=1).to(device)

# Normalize
xy_colloc_norm = normalize(xy_colloc, torch.tensor([0, -B_half], device=device), torch.tensor([L, B_half], device=device))
xy_bc_left_norm = normalize(xy_bc_left, torch.tensor([0, -B_half], device=device), torch.tensor([L, B_half], device=device))
xy_bc_right_norm = normalize(xy_bc_right, torch.tensor([0, -B_half], device=device), torch.tensor([L, B_half], device=device))
xy_bc_wall_norm = normalize(xy_bc_wall, torch.tensor([0, -B_half], device=device), torch.tensor([L, B_half], device=device))
xy_bc_con_norm = normalize(xy_bc_con, torch.tensor([0, -B_half], device=device), torch.tensor([L, B_half], device=device))

# Model and optimizer (udamos ADAM)
model = PINN2D().to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=2000, gamma=0.5)

# En caso de usar uno precargado
# model.load_state_dict(torch.load('pinn_trained.pth'))

# Entrenamiento
n_steps = 7000 # numero de epocas
for step in range(n_steps):
    optimizer.zero_grad()
    loss = loss_function(model, xy_colloc_norm, xy_bc_left_norm, xy_bc_right_norm, xy_bc_wall_norm, xy_bc_con_norm)
    loss.backward()
    torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=0.5)
    optimizer.step()
    scheduler.step()
    if step % 500 == 0:
        print(f"Step {step}, Loss: {loss.item():.6f}")

# Guardamos el modelo ya entrenado
torch.save(model.state_dict(), 'pinn_trained.pth')

# Resultados
with torch.no_grad():
    pred = model(xy_colloc_norm)
    h = pred[:, 0].cpu().reshape(N, N)
    u = pred[:, 1].cpu().reshape(N, N)
    zb = pred[:, 2].cpu().reshape(N, N)
    v = torch.zeros_like(u)  # v = 0

# Extraigo la velocidad en diferentes puntos x = 0, 2.7, 3.4 m
N_y = 100
y_cross = torch.linspace(-B_half, B_half, N_y).to(device)
x_positions = [0.0, 2.7, 3.4]
u_cross_sections = []

with torch.no_grad():
    for x_pos in x_positions:
        x_cross = torch.full((N_y,), x_pos).to(device)
        xy_cross = torch.stack([x_cross, y_cross], dim=1)
        xy_cross_norm = normalize(xy_cross, torch.tensor([0, -B_half], device=device), torch.tensor([L, B_half], device=device))
        pred_cross = model(xy_cross_norm)
        u_cross = pred_cross[:, 1].cpu().numpy()
        u_cross_sections.append(u_cross)


###################################################################################
## PLOT
###################################################################################

# Plot cross-sections
plt.figure(figsize=(10, 6))
for i, x_pos in enumerate(x_positions):
    plt.plot(y_cross.cpu().numpy(), u_cross_sections[i], label=f'x = {x_pos} m')
plt.axvspan(-0.74, -0.44, color='gray', alpha=0.3, label='Estribo (x = 2.94–3.06 m)')
plt.xlabel('y [m]')
plt.ylabel('Velocidad u [m/s]')
plt.title('Cortes Transversales de Velocidad en x = 0, 2.7, 3.4 m')
plt.legend()
plt.grid(True)
plt.savefig('velocity_cross_sections_flow_continuity.png')
plt.close()

# Quiver plot for velocity field
fig = plt.figure(figsize=(10, 3))
plt.quiver(X.cpu(), Y.cpu(), u, v, color='blue', scale=10)
plt.title('Campo de Velocidades (u, v) en Estado Estacionario')
plt.xlabel('x [m]')
plt.ylabel('y [m]')
plt.fill_betweenx(np.linspace(-0.44, -0.74, 10), 2.94, 3.06, color='gray', alpha=0.5)
plt.savefig('velocity_field_flow_continuity.png')
plt.close()

# Contour plots for h, u, zb
fig, axs = plt.subplots(3, 1, figsize=(12, 8))
cf1 = axs[0].contourf(X.cpu(), Y.cpu(), h, levels=20, cmap='Blues')
fig.colorbar(cf1, ax=axs[0])
axs[0].set_title('Altura (h)')
axs[0].fill_betweenx(np.linspace(-0.44, -0.74, 10), 2.94, 3.06, color='gray', alpha=0.5)

cf2 = axs[1].contourf(X.cpu(), Y.cpu(), u, levels=20, cmap='Reds')
fig.colorbar(cf2, ax=axs[1])
axs[1].set_title('Velocidad (u)')
axs[1].fill_betweenx(np.linspace(-0.44, -0.74, 10), 2.94, 3.06, color='gray', alpha=0.5)

cf3 = axs[2].contourf(X.cpu(), Y.cpu(), zb, levels=20, cmap='terrain')
fig.colorbar(cf3, ax=axs[2])
axs[2].set_title('Elevación del Fondo (zb)')
axs[2].fill_betweenx(np.linspace(-0.44, -0.74, 10), 2.94, 3.06, color='gray', alpha=0.5)

for ax in axs:
    ax.set_xlabel('x [m]')
    ax.set_ylabel('y [m]')
    ax.set_aspect('auto')

plt.tight_layout()
plt.savefig('contour_plots_flow_continuity.png')
plt.close()

Using: cpu
Step 0, Loss: 73.300751
Step 500, Loss: 0.191747
Step 1000, Loss: 0.145293
Step 1500, Loss: 0.097153
Step 2000, Loss: 0.061547
Step 2500, Loss: 0.048820
Step 3000, Loss: 0.041189
Step 3500, Loss: 0.035635
Step 4000, Loss: 0.030982
Step 4500, Loss: 0.029234
Step 5000, Loss: 0.027777
Step 5500, Loss: 0.026459
Step 6000, Loss: 0.025502
Step 6500, Loss: 0.024192
